In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import asyncio
import boto3
import aioboto3
from dotenv import load_dotenv, find_dotenv
import random

In [8]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# Encontra o arquivo .env no diretório pai
dotenv_path = find_dotenv()

# Carrega as variáveis de ambiente do arquivo .env encontrado
load_dotenv(dotenv_path)

True

In [9]:
dataset_peoples = pd.read_csv('../data/peoples.csv')
dataset_pets = pd.read_csv('../data/dataset_union.csv')

In [10]:
uuids_peoples = dataset_peoples['ID'].to_list()
ids_pets = dataset_pets['ID'].to_list()

In [11]:
from collections import namedtuple

Pet = namedtuple('Pet', ['id', 'animal_name', 'animal_gender', 'breed_name', 'borough', 'zip_code', 'type', 'image'])

In [12]:
print(len(dataset_pets))

134291


In [13]:
dict_people_pet = {}

# garantindo que todo usuário terá um animal
for index, id in tqdm(enumerate(uuids_peoples), total=len(uuids_peoples)):
    pet = dataset_pets.loc[dataset_pets.index[index]]
    dict_people_pet[f"{id}"] = [Pet(id = pet['ID'], animal_name = pet['AnimalName'], animal_gender = pet['AnimalGender'], breed_name = pet['BreedName'], borough = pet['Borough'],  zip_code = str(pet['ZipCode']), type = pet['Type'], image = pet['Image'])]
    # if index in dataset_pets.index:
    #     dataset_pets = dataset_pets.drop(index)

100%|██████████| 24500/24500 [00:00<00:00, 32219.07it/s]


In [16]:
count_pets = len(ids_pets)
count_users = len(uuids_peoples)

dict_people_pet_final = {}
people_keys = list(uuids_peoples)
random.shuffle(people_keys)

with tqdm(total=count_pets, desc=f"Handing out pets...") as inner_tqdm:
    for person_id in people_keys:
        quantity_pets = np.random.randint(1, 9)

        # Gera aleatoriamente uma lista de índices de animais
        pet_indices = np.random.choice(dataset_pets.index, size=quantity_pets, replace=False)
        person_pets = []

        for pet_index in pet_indices:
            pet = dataset_pets.loc[pet_index]
            person_pets.append(Pet(id=pet['ID'], animal_name=pet['AnimalName'], animal_gender=pet['AnimalGender'], breed_name=pet['BreedName'], borough=pet['Borough'], zip_code=str(pet['ZipCode']), type=pet['Type'], image=pet['Image']))

        dict_people_pet_final[person_id] = person_pets
        inner_tqdm.update(quantity_pets)

        if len(dataset_pets) < quantity_pets:
            break

        # Marca os animais que foram distribuídos como indisponíveis
        dataset_pets = dataset_pets.drop(index=pet_indices)

print("Done handing out pets!")

Handing out pets...:  58%|█████▊    | 77721/134291 [01:23<01:00, 933.73it/s] 

Done handing out pets!


In [17]:
session = aioboto3.Session(
    aws_access_key_id       = os.getenv('AWS_ACCESS_KEY_ID'),  
    aws_secret_access_key   = os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name             = os.getenv('AWS_DEFAULT_REGION'), 
    aws_session_token       = os.getenv('AWS_ACCESS_TOKEN')
)

In [18]:
s3 = session.resource("s3")

/var/folders/9p/bjctjlpd1c9dk82v6v2nqmdh0000gn/T/ipykernel_52465/2104096300.py:1: RuntimeWarning: coroutine 'AioSession._create_client' was never awaited
  s3 = session.resource("s3")


In [29]:
async def directory_exists(bucket_name, directory_name):
    try:
        async with session.resource("s3") as s3:
            bucket = await s3.Bucket(bucket_name)
            await bucket.head_object(Bucket=bucket_name, Key=directory_name)
        return True
    except:
        return False

In [22]:
import os

file_list = []
dir_atual = os.getcwd()
dir_destino = os.path.abspath(f'{os.pardir}/inteligencia_artificial/train')

os.chdir(dir_destino)

arquivos_jpg = [f for f in os.listdir() if f.endswith('.jpg')]
fotos = set(arquivos_jpg)

In [23]:
len(fotos)

75501

In [30]:
bucket_name = 'pilha-nuvem-tcc-sptech-bucket'

async def send_image_to_s3(md5_hash, folder, pet_name):
    image_with_extension = f"{md5_hash}.jpg"
    image_path = f"{dir_destino}/{image_with_extension}"
    data = open(image_path, 'rb')
    async with session.resource("s3") as s3:
        bucket = await s3.Bucket(bucket_name)
        await bucket.put_object(Key = f"{folder}/{pet_name}_{image_with_extension}", Body=data)

In [31]:
for user_id, pets in tqdm(dict_people_pet_final.items()):
    user_folder = f"train/{user_id}"
    if not await directory_exists(bucket_name = bucket_name, directory_name = user_folder):
        for pet in pets:
            if pet.image:
                await send_image_to_s3(pet.image, f"{user_folder}/{pet.id}", pet.animal_name)

  0%|          | 38/17251 [03:23<25:37:36,  5.36s/it]


CancelledError: 